<a href="https://colab.research.google.com/github/jiztastamablastamarang/TwitterSturm/blob/main/TwitterUser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##WebDriver##

###Modules###

In [122]:
!pip install -U -q selenium
!pip install -U -q snscrape

!apt-get -qq update
!apt install -q chromium-chromedriver
!dpkg --configure -a

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.3 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists...
Building dependency tree...
Reading state information...
chromium-chromedriver is already the newest version (99.0.4844.51-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [123]:
import pandas as pd
import requests
import os
import glob
import random
import shutil
import time
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from IPython.display import Image
import snscrape.modules.twitter as sns


In [124]:
def screenshot(webdriver):
  t=3
  time.sleep(t)
  webdriver.fullscreen_window()
  img = webdriver.get_screenshot_as_png()
  return Image(img)

###TwitterSoldier Class###

In [132]:
from collections import UserList
class TwitterSoldier:

    def __init__(self, phone, password, username):
      self.email = phone
      self.password = password
      self.nick = username
      t = 10
      options = Options()
      options = webdriver.ChromeOptions()
      options.add_argument("--start-maximized")
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument('--disable-dev-shm-usage')
      options.add_argument("--window-size=3920,2080")
      options.add_argument("--blink-settings=imagesEnabled=false")
      self.driver = webdriver.Chrome(executable_path="chromedriver", options=options)
      self.waiter = WebDriverWait(self.driver, t)
      self.action = ActionChains(self.driver)
      

    def login(self):
      messedge = "We've texted you a login verification code"
      driver = self.driver
      wait = self.waiter
      driver.get("https://twitter.com/login")
      time.sleep(3)

      def enter(text):
        field = wait.until(EC.presence_of_element_located((By.TAG_NAME, "input")))
        field.send_keys(text)
        field.send_keys(Keys.ENTER)
        time.sleep(3)

      enter(self.email)
      enter(self.password)

      if driver.page_source.count(messedge):
        print(messedge)
        text = input()
        enter(text)
      # try:
      #   enter(self.nick)
      # except:
      #   pass

      return screenshot(driver)

    def reply_status(self, url, status, image):
        driver = self.driver
        wait = self.waiter
        action = self.action 
        driver.get(url)
        try:
          reply = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@data-contents='true']")))
          reply.send_keys(status)
          if image:
            media = driver.find_element(By.XPATH, "//input[@data-testid='fileInput']")
            media.send_keys(image)
          button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='tweetButtonInline']")))
          button.click()
          print(url)
          # return screenshot(driver)
        except:
          print(f"error sending reply to status: {url}")
          return url


    def user_tweets(self, user):
      driver = self.driver
      wait = self.waiter
      action = self.action 
      driver.get(f"https://twitter.com/{user}")
      for i in range(5):
        action.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
      urls = []
      tweets = driver.find_elements(by=By.TAG_NAME, value="a")
      for tweet in tweets:
          url = tweet.get_attribute("href")
          if url.lower().count(f"{user.lower()}/status") and not url.count("photo"):
            urls.append(url)
      return urls     



##Docs##
Гуглотаблиці списком користувачей, контентом, директорія з фото

In [126]:
USERS = "https://docs.google.com/spreadsheets/d/1uUF6WGtFl-WBPe_oLdk4gdgFp09aPOAekBWmHdIUnC0/edit#gid=0" #повний список сенаторів
CONTENT = "https://docs.google.com/spreadsheets/d/1dsdTocyIgQpJYrBLnjxBl9e3Bdw65vAWmpBPorzLlrI/edit#gid=0"# коменти
IMGS = "https://drive.google.com/drive/folders/13KrHYesxtF-Ut21EZe1ScOku8-wm-5JY" # директорія з фото

###Images###
Може бути запрошений код підтрвердження

In [ ]:
# folder_id = IMGS.split("/")[-1]
# images = []

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# path = "./temp"
# try: shutil.rmtree(path)
# except: pass

# try: os.makedirs(path)
# except: pass

# file_list = drive.ListFile(
#     {'q': "'{}' in parents".format(folder_id)}).GetList()

# for image in file_list:
#     fname = os.path.join(path, image["title"])
#     new_file = drive.CreateFile({"id": image["id"]})
#     new_file.GetContentFile(fname)
#     print("uploading: " + image["title"])
#     images.append(os.path.join("/content", "temp", image["title"] ))

uploading: 274791836_1368383970268989_7318621539946358037_n.png
uploading: WhatsApp Image 2022-03-04 at 13.38.41.jpeg


###Users###

In [120]:
def load_doc(url):
  return pd.read_csv(url.replace("/edit#gid=", "/export?format=csv&gid="), on_bad_lines="skip")

In [121]:
users = load_doc(USERS)
try:
  users["TwitterNickname"] = users["Twitter"].str.replace("https://twitter.com/", "", regex=False).str.replace("@", "", regex=False)
except:
  pass
users.dropna(subset=["TwitterNickname"], inplace=True)
users

,State,Member of Congress,Name,Party,Twitter,Facebook,TwitterNickname
0,Alabama,U.S. Senator,"Shelby, Richard",R,@SenShelby,x,SenShelby
1,Alabama,U.S. Senator,"Tuberville, Tommy",R,@Ttuberville,x,Ttuberville
2,Alabama 1st District,U.S. Representative,"Carl, Jerry",R,@RepJerryCarl,x,RepJerryCarl
3,Alabama 2nd District,U.S. Representative,"Moore, Barry",R,@RepBarryMoore,x,RepBarryMoore
4,Alabama 3rd District,U.S. Representative,"Rogers, Mike",R,@RepMikeRogers,x,RepMikeRogers
...,...,...,...,...,...,...,...
536,Wisconsin 7th District,U.S. Representative,"Tiffany, Thomas",R,@TomTiffanyWI,x,TomTiffanyWI
537,Wisconsin 8th District,U.S. Representative,"Gallagher, Mike",R,@MikeforWI,x,MikeforWI
538,Wyoming,U.S. Senator,"Lummis, Cynthia",R,@SenLummis,x,SenLummis
539,Wyoming,U.S. Senator,"Barrasso, John",R,@SenJohnBarrasso,x,SenJohnBarrasso


###Content###

In [86]:
content = load_doc(CONTENT)
content

,Messedge,Lengh
0,This story is fake and part of propaganda. Rus...,183


##Credentials & login##

In [ ]:
PHONE = "+380994404244"
USERNAME = "jiztastamablast"
PASSWORD = "6pnpd7pg"

bot = TwitterSoldier(PHONE, PASSWORD, USERNAME)
bot.login()

##Twitter Scrapper##

In [81]:
tweets_list = []
query = "Ovsyannikova"
end = datetime.now().strftime("%Y-%m-%d")
start = (datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d")

for i, tweet in enumerate(sns.TwitterSearchScraper(f"{query} since:{start} until:{end}").get_items()):
    tweets_list.append(tweet.url)

In [109]:
print(f"Знайдено {len(tweets_list)} повідомлень")

Знайдено 15380 повідомлень


In [93]:
for tweet in tweets_list:
  for user in users["TwitterNickname"].to_list():
    if tweet.lower().count(user.lower()):
      print(tweet)

https://twitter.com/boblatta/status/1503739608047177732
https://twitter.com/RepMcGovern/status/1503526356642148355
https://twitter.com/SenSanders/status/1503504857449353226


##Main##

In [ ]:
shots = []

for url in list(reversed(tweets_list)):
  status= random.choice(content["Messedge"].to_list())
  media = None
  shots.append(bot.reply_status(url, status, media))